# Intro to Feature Stores

## Introduction
>Feature stores are central hubs for feature storage. In other words, they allow users to transform raw data into feature values, store these values, and use them for (re)training a model or making predictions. 

In contrast to classic databases, feature stores are beneficial particularly for addressing challenges that surface when working with large volumes of data across many teams.

### Possible challenges
- Different teams may be using the same name for slightly different features.
- Different teams may be computing a feature in different and potentially incorrect ways.
- Computed features required by a model may not be available immediately when required to make predictions in production.
- Features may be computed differently and incorrectly between training and serving (deployment) phases.
- Features may be computed differently and incorrectly between development, testing and production phases.
- Different teams may not have access to the central, correct feature store.

#### Example
If you are working with a considerably large dataset, the computation speed of your engineered features will decrease significantly because of the large volume of data to be processed. For example, consider that you have engineered a useful feature, 'Average purchase value'. If you have millions of purchases, this feature will be difficult to compute because each time you conduct the computation, you must process all the data. As a solution, you could store a running feature so that the next time you add data, you can access the feature store and retrieve the feature without processing the whole dataset.

<p><img src=images/Feature_Stores_API.png></p>

## Benefits of Feature Stores
Feature stores offer the following advantages:
<details>
  <summary> 
    <b>1. Use features without needing to know and check how they are computed.</b>
  </summary>

  Most features depend on a couple of operations that are not very specific to a field. However, some features may depend on a team of engineers that have a profound knowledge of the field and have created complicated formulas for computing them. <br><br>
  
</details>


<details>
  <summary> 
    <b> 2. Automate feature computation, backfills and logging.</b>
  </summary>

  When data are ingested, features need to be recomputed. Feature stores can automatically compute the new value. They can also fill in computed feature values in historical data entries. This is called 'backfilling'.<br><br>
</details>

<details>
  <summary> 
    <b> 3. Share and reuse features across teams</b>
  </summary>

  Features created by the engineering team can be used by the data-science team and vice-versa. The features you create may help the engineering team ascertain the usefulness of the pipeline they are developing.<br><br>
</details>

<details>
  <summary> 
    <b> 4. Achieve consistency between training and serving data.</b>
  </summary>

Features have to be computed consistently between the training phase and post-deployment phase. If not, the performances may be different. A common pitfall is the use of an undocumented approach to compute features during training, which may cause problems in production.<br><br>
</details>

<details>
  <summary> 
    <b> 5. Monitor the health of feature pipelines in production.</b>
  </summary>

  After registering features in a feature store, you will be able to confirm if the pipeline they are used in is producing good results. By tracking the feature store, you can easily observe which feature works and which does not when linked to the outcomes.<br><br>
</details>


## Offline vs Online Features

As you are aware, a feature is an input variable. However, there are differences between _Offline_ and _Online_ features.

### Offline features
These are static, historic features that do not change much and are processed in batches. Generally, offline features are calculated using frameworks, such as Spark, or by running SQL queries against a given database and using a batch inference process thereafter. These features can be properties (e.g. a patient's age), a transaction location or an IP address.<br><br>

### Online features
These features are dynamic and require a processing engine to calculate, sometimes in near-real-time. They often need to be served in ultra-low latency. Any feature that provides an average over some history, for example, is an online feature. Another example is calculating a z-score for real-time fraud detection. In this case, the pipeline is built by calculating the mean and the standard deviation over a sliding window in real-time. These calculations are highly challenging, and they require high computation speeds, as well as fast access to the data. The data can be stored in the memory or in a very fast key-value database.

An even more complicated type can be engineered by using an ML process to generate the feature from a data point, e.g. creating a 'contains a positive product review' feature by utilising NLP to identify positive reviews of a product.

## Feature Store Components

There are five main components/functions of a modern feature store: transformation, storage, serving, monitoring, and feature registry.

### Serving

Feature stores serve feature data to models. These models require a consistent view of features across training and serving. The definitions of features used to train a model must exactly match the features provided in online serving. If not, a training-serving skew is introduced, which can cause catastrophic and hard-to-debug model-performance problems. This may occur when different teams compute features using slightly different approaches or when one forgets to normalise data.

Feature stores abstract away the logic and processes required to generate a feature, providing users with an easy and canonical approach for accessing all the features in a company consistently across all environments in which they are needed.

When retrieving data offline (i.e. for training), feature values are commonly accessed through notebook-friendly feature-store SDKs. They provide point-in-time correct views of the state of the world for each example used to train a model (i.e. 'time travel').
For online serving, a feature store delivers a single vector of features at a time, comprising the latest feature values. Responses are served through a high-performance API backed by a low-latency database.

### Storage

Feature stores enable feature-data retrieval through feature serving layers. Typically, they contain online and offline storage layers to support the requirements of different feature-serving systems.

Offline storage layers are typically used to store months’ or years’ worth of feature data for training. Offline feature-store data are often stored in data warehouses or data lakes, such as S3, BigQuery, Snowflake and Redshift. Extending an existing data lake or data warehouse for offline-feature storage is typically preferred to prevent the use of data silos.

Online storage layers are used to persist feature values for low-latency lookup during inference. They typically only store the latest feature values for each entity, essentially modeling the current state of the world. Online stores are generally eventually consistent, and they do not have strict consistency requirements for most ML use cases. They are usually implemented with key-value stores, such as DynamoDB, Redis or Cassandra.

<p align=center><img src=images/Feature_Store_Storage.png width=500></p>

Feature stores use an entity-based data model, where each feature value is associated with an entity (e.g. a user) and a timestamp. An entity-based data model provides minimal structure to support standardised feature management, fits naturally with common feature engineering workflows and allows for simple retrieval queries in production.

### Transformation

Operational ML applications require regular processing of new data into feature values so that models can make predictions using an up-to-date view of the world. Feature stores both manage and orchestrate data transformations that produce these values, as well as ingest values produced by external systems. The transformations managed by feature stores are configured by definitions in a common feature registry.

_Most teams getting started with feature stores already have existing data pipelines producing feature values. As such, it is very important for feature stores to be gradually adaptable and have first-class integrations with existing data platforms. This way, teams can immediately operationalise existing ETL pipelines for their ML use cases._

| Feature Type 	| Definition 	| Example 	|
|---	|---	|---	|
| Batch transform 	| Transformations that are applied only to data at rest. 	| User country, product category. 	|
| Streaming transform 	| Transformations that are applied to streaming sources. 	| Number of clicks per vertical per user in the last 30 minutes. Number of views per listing in the past hour.	|
| On-demand transform 	| Transformations used to produce features based on data that are only available at the time of the prediction. These features cannot be pre-computed. 	| Is the user currently in a supported location? The similarity score between listing and search query. 	|

Models require access to the latest feature values for inference. Feature stores accomplish this by regularly recomputing features on an ongoing basis. Transformation jobs are orchestrated to ensure that new data is processed and converted into new feature values. These jobs are executed on data-processing engines (e.g. Spark or Pandas) to which the feature store is connected. 

Model development introduces different transformation requirements. When iterating over a model, new features are often engineered for use in training datasets that correspond to historical events (e.g. all purchases in the last 6 months). To support these use cases, feature stores simplify the running of 'backfill jobs' that generate and persist historical values of a feature for training. Some feature stores automatically backfill newly registered features for preconfigured time ranges for registered training datasets.

Transformation code is reused across environments, which prevents training-serving skew and frees teams from having to rewrite code from one environment to the next.

An example of a feature store containing the already mentioned three components is shown below:

<p align=center> <img src=images/Feature_Stores_TSS.png> </p>

As shown in the figure, feature management involves Transformation, and access can be considered as Serving.

### Monitoring

When a failure occurs in an ML system, it is usually a data problem. Feature stores are uniquely positioned to detect and surface such issues. They can calculate metrics on the features they store and serve that describe correctness and quality. Feature stores monitor these metrics to provide a signal of the overall health of an ML application.

Feature data can be validated based on user-defined schemas or other structural criteria. Data quality is tracked by monitoring for drift and training-serving skew. For example, feature data served to models are compared to data on which the model was trained to detect inconsistencies that could degrade the model performance.

When running production systems, it is also important to monitor operational metrics. Feature stores track operational metrics relating to core functionality, e.g. metrics relating to feature storage (availability, capacity, utilisation, staleness) or feature serving (throughput, latency, error rates). Other metrics describe the operations of important adjacent system components, e.g. operational metrics for external data-processing engines (e.g. job success rate, throughput, processing lag and rate).

Feature stores make these metrics available to existing monitoring infrastructure. This allows the health of ML applications to be monitored and managed with existing observability tools in the production stack.

Having visibility into which features are used by which models, feature stores can automatically aggregate alerts and health metrics into views relevant to specific users, models or consumers.

### Registry

A critical component of all feature stores is a centralised registry of standardised feature definitions and metadata. The registry acts as a single source of truth for information about a feature in an organisation. 

The registry is a central interface for user interactions with the feature store. Teams use the registry as a common catalog to explore, develop, collaborate on and publish new definitions within and across teams.

The definitions in the registry configure feature-store system behavior. Automated jobs use the registry to schedule and configure data ingestion, transformation and storage. It forms the basis of what data are stored in the feature store and how they are organised. Serving APIs use the registry for a consistent understanding of which feature values should be available, who should be able to access them and how should they be served.

The registry allows for important metadata to be attached to feature definitions. This provides a route for tracking ownership, project or domain-specific information, as well as a path to easily integrate with adjacent systems. This includes information about dependencies and versions, which is used for lineage tracking. 

To help with common debugging, compliance and auditing workflows, the registry acts as an immutable record of what is available analytically and what is actually running in production.

Thus far, we have explored the core minimal components of a feature store. In practice, companies often have needs, such as compliance, governance and security, that require additional enterprise-focused capabilities. This will be covered in a future lesson. 

## Feature Store Providers

Here, we provide a non-exhaustive list of commercial feature stores, along with brief descriptions.

- __Feast__: Feast has grown in popularity recently, and it is currently one of the most used feature store providers globally. It works seamlessly with Python, and the library can handle many file-management intricacies. This is a great option for users that have already created the transformation pipeline.
- __Tecton__: Tecton is another popular feature store provider that includes end-to-end pipelines for features. This, as opposed to Feast, supports transformations.
- __AWS__: Amazon does not actually provide a stand-alone feature store service. Instead, it integrates this service in its SageMaker service.
- __Hospworks__: Hospworks supports the full stack mentioned in this lesson, and it is equipped with a clean UI. Additionally, it can integrate more third-party services, enabling the easy scaling of pipelines.
- __Iguazio__: This feature store has a tailored platform and does not rely on other platforms to store features. Additionally, it supports third-party services.

For in-depth information on each provider and to compare them, visit this [link](https://mlops.community/learn/feature-store/).

In this module, we will focus on Feast because it is easy to integrate, has a large community, and is a great way to 'get started' with feature stores.

## Conclusion
At this point, you should have a good understanding of 

- feature stores and their benefits.
- differences between offline and online feature stores.
- feature store components and providers.